In [52]:
import sounddevice as sd
import numpy as np
import librosa
import wave
import os
import joblib
from tensorflow.keras.models import load_model
import time
from pydub import AudioSegment
from pydub.playback import play


# Function to record audio from the microphone
def record_audio(duration=5, sample_rate=16000):
    """
    Records audio from the local PC microphone for a specified duration.
    
    Args:
        duration (int): Duration of the recording in seconds.
        sample_rate (int): Sampling rate for the audio.
        
    Returns:
        np.array: The recorded audio signal as a NumPy array.
    """
    print(f"🎙️ Recording for {duration} seconds...")
    # Record the audio, the duration * sample_rate determines the number of samples
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='int16')
    sd.wait()  # Wait until the recording is finished
    print("📢 Recording complete!")
    
    # Convert the audio data to floating-point format (librosa expects float32)
    return recording.astype(np.float32)

# Function to save the recorded audio to a .wav file
def save_audio(recording, filename='output.wav', sample_rate=16000):
    """
    Saves the recorded audio to a .wav file.
    
    Args:
        recording (np.array): The recorded audio signal as a NumPy array.
        filename (str): The name of the file to save the audio.
        sample_rate (int): Sampling rate for the audio.
    """
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(1)  # Mono audio
        wf.setsampwidth(2)  # 16-bit audio
        wf.setframerate(sample_rate)  # Sample rate
        wf.writeframes(recording.tobytes())  # Write the audio data to the file
    print(f"✅ Audio saved as {filename}")

# Load the pre-trained model and scaler (using joblib here)
model_path = r'C:\Users\dell\Data Science\TensorFlow Project\model.joblib'
scaler_path = r'C:\Users\dell\Data Science\TensorFlow Project\scaler.joblib'

# Check if the files exist
if os.path.isfile(model_path):
    print(f"✅ Model file exists at {model_path}")
else:
    print(f"❌ Model file not found at {model_path}")

if os.path.isfile(scaler_path):
    print(f"✅ Scaler file exists at {scaler_path}")
else:
    print(f"❌ Scaler file not found at {scaler_path}")

# Load model and scaler
model = joblib.load(model_path)
scaler = joblib.load(scaler_path)

print("✅ Model and Scaler loaded successfully!")

def extract_features(filename):
    # Load audio file with librosa
    y, sr = librosa.load(filename, sr=None)  # sr=None to keep the original sampling rate
    # Extract MFCCs (Mel-Frequency Cepstral Coefficients)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # 13 MFCC coefficients
    # Take the mean of each coefficient (feature aggregation)
    mfcc_mean = np.mean(mfcc, axis=1)
    return mfcc_mean


# Function to predict emotion from the extracted features
def predict_emotion(filename='output.wav'):
    """
    Predicts if the audio is 'Crying' or 'Laughing' using the pre-trained model.
    
    Args:
        audio (np.array): The raw audio signal as a NumPy array.
        
    Returns:
        str: The predicted label (Crying or Laughing).
    """
    # Step 1: Extract MFCC features from the audio
    mfcc_features = extract_features(filename='output.wav')
    
    # Reshape the features to match the model input
    mfcc_features = mfcc_features.reshape(1, -1)  # Ensure the shape is (1, 13)
    
    # Step 2: Scale the features using the loaded scaler
    mfcc_features_scaled = scaler.transform(mfcc_features)
    
    # Step 3: Make predictions using the model
    prediction = model.predict(mfcc_features_scaled)
    
    # Map the model's prediction to a label (Crying or Laughing)
    predicted_label = 'Crying' if prediction[0] == 1 else 'Laughing'
    
    return predicted_label

from pydub import AudioSegment
from pydub.playback import play

def reduce_noise(audio_file, output_file):
    # Load audio file
    audio = AudioSegment.from_wav(audio_file)
    
    # Use a noise reduction method like this (simplified example)
    # Here, you can play with the parameters for better results
    audio = audio.low_pass_filter(3000)  # Remove high frequencies
    
    # Save the filtered audio
    audio.export(output_file, format="wav")
    print(f"✅ Audio saved after noise reduction to {output_file}")
    
# Main script
# Record audio for 3 seconds
audio_data = record_audio(duration=3, sample_rate=16000)

# Save the recorded audio to a .wav file
save_audio(audio_data, filename='output.wav', sample_rate=16000)

# Reduce noise and save the filtered audio
reduce_noise('output.wav', 'output_filtered.wav')

# Play the original recorded audio
print("🎧 Playing back the recorded audio...")
sd.play(audio_data, samplerate=16000)
sd.wait()  # Wait for playback to finish
print("✅ Playback complete!")

# Play the filtered audio
print("🎧 Playing back the filtered audio...")
filtered_audio_data, _ = librosa.load('output_filtered.wav', sr=16000)  # Load the filtered audio file
sd.play(filtered_audio_data, samplerate=16000)  # Play the filtered audio
sd.wait()  # Wait for playback to finish
print("✅ Playback complete!")

# Predict emotion from the filtered audio
predicted_label = predict_emotion('output_filtered.wav')
print(f"🎉 Predicted Emotion: {predicted_label}")

✅ Model file exists at C:\Users\dell\Data Science\TensorFlow Project\model.joblib
✅ Scaler file exists at C:\Users\dell\Data Science\TensorFlow Project\scaler.joblib
✅ Model and Scaler loaded successfully!
🎙️ Recording for 3 seconds...
📢 Recording complete!
✅ Audio saved as output.wav
✅ Audio saved after noise reduction to output_filtered.wav
🎧 Playing back the recorded audio...
✅ Playback complete!
🎧 Playing back the filtered audio...
✅ Playback complete!
🎉 Predicted Emotion: Crying
